In [2]:
# データ前処理1（dataframeで必要な列と行の取り出し）
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
import ast

# 入力TSVファイルのパスを指定します
input_tsv_name = 'FullAuto_touch_result__of__4m_45_1___20240510-130005.tsv.txt'
input_tsv_path = './data/input/' + input_tsv_name
# TSVファイルを読み込みます
df = pd.read_csv(input_tsv_path, sep='\t')

# HeatMap列が存在するかのflagデータを追加
df['hm'] = df['Heat Map Points'].isna().apply(lambda x: 0 if x else 1)

# 必要な行だけ抽出
needed_row = ['Person index', '3D KeyPoints', 'hm']
df = df[needed_row]

# 欠損列を削除
df = df.dropna()

# 欠損列を除いた場合のデータ数
print(len(df))

# 行数を振り直し
df = df.reset_index(drop=True)

# 最初の５行を出力
print(df.head())

# 動画のフレームレート
# FIXME: fpsが奇数の場合（例: 15fps）に対応できていない 
fps = 30

# 0.5sごとに分割（つまり、2fpsにする）
df_2fps = df[::int(fps/2)]

for row in df_2fps.values:
    print(row)

# 2fpsに変更後のCSVを出力
df_2fps.to_csv('data/tmp/2fps/' + input_tsv_name, index=False)

263
  Person index                                       3D KeyPoints  hm
0          [2]  [[[3072.6642238096324, -319.86854124266165, 18...   0
1          [2]  [[[2418.6999311121444, -689.6644450512248, 146...   0
2          [2]  [[[2128.173189443695, -861.795713268067, 1432....   0
3          [2]  [[[1850.5269163174446, -938.8016819391878, 138...   1
4          [2]  [[[1712.1461353290094, -977.493400787035, 1325...   1
['[2]'
 '[[[3072.6642238096324, -319.86854124266165, 1868.012393682472], [2585.8643526964465, -253.81794234545484, 2303.449351297077], [2350.1322856425236, -400.4605899543724, 2188.2681865144064], [1647.0560800943417, -751.5505678298773, 2329.1253966814643], [1448.3393184038694, -703.260744162903, 2253.5809817047684], [2646.8698174603883, -141.7163197897167, 2244.083294955568], [2076.96500808918, -165.31063853899795, 2383.5518414593557], [2239.53118081571, 171.55261709350063, 2400.4814667595824], [1629.0368838633642, -652.3104579408706, 2280.1928586326458], [2105.625980

In [3]:
# データ前処理2（データの型変換）

# CSVファイルを読み込みます
df = pd.read_csv('./data/tmp/2fps/' + input_tsv_name, sep=',')

# 最初の５行を出力
print('変更前df')
print(df.head())

# 'Person index'列の修正
df['Person index'] = df['Person index'].str.strip('[]').astype(int)

# '3D KeyPoints'列の修正
# '3D KeyPoints'列の文字列をPythonリストに変換し、その上で三次元配列から二次元配列に変換
df['3D KeyPoints'] = df['3D KeyPoints'].apply(lambda x: ast.literal_eval(x)[0])
# Pythonリストをnumpy配列に変換
df['3D KeyPoints'] = df['3D KeyPoints'].apply(np.array)

# 最初の５行を出力
print('変更後df')
print(df.head())

# 型が正しく変更されたか確認
print(type(df['3D KeyPoints'].iloc[0]))
print(df['3D KeyPoints'].iloc[0][0][1])
print(type(df['3D KeyPoints'].iloc[0][0][1]))

変更前df
  Person index                                       3D KeyPoints  hm
0          [2]  [[[3072.6642238096324, -319.86854124266165, 18...   0
1          [2]  [[[1307.8577889330436, -1785.2186370120187, 20...   0
2          [2]  [[[1554.4777572780208, -1893.913998428577, 179...   0
3          [2]  [[[1512.1017197671745, -1859.5323782830799, 11...   0
4          [2]  [[[1481.7128677928902, -1828.6531721275294, 43...   0
変更後df
   Person index                                       3D KeyPoints  hm
0             2  [[3072.6642238096324, -319.86854124266165, 186...   0
1             2  [[1307.8577889330436, -1785.2186370120187, 209...   0
2             2  [[1554.4777572780208, -1893.913998428577, 1796...   0
3             2  [[1512.1017197671745, -1859.5323782830799, 114...   0
4             2  [[1481.7128677928902, -1828.6531721275294, 435...   0
<class 'numpy.ndarray'>
-319.86854124266165
<class 'numpy.float64'>


In [4]:
# 状態判定
from matplotlib.path import Path

# 「移動」→ 1,「カートに追加」→ 2,「会計」→ 3,「ウェイト」→ 4とする
state_dict = {"移動": 1, "カートに追加": 2, "会計": 3, "ウェイト": 4, "未設定": -1}

# i.「カートに追加」の判定
df['state'] = df['hm'].apply(lambda x: state_dict['カートに追加'] if x == 1 else state_dict['未設定'])


# ii. 「会計」の判定
# レジ前ゾーン（長方形）の定義
# FIXME: 適当な値をおいている状態
# cashier_zone = [[630, -80] , [950, -80], [950, 50], [630, 50]]
cashier_zone = [[0, 0] , [0, 200], [200, 200], [200, 0]]

# 点が長方形の内部にあるか判定する関数
def is_point_in_square(square, point):
    path = Path(square)  # 四角形のPathオブジェクトを作成
    return path.contains_point(point) # pointがPath内部に含まれるか判定

# 両足のつま先がレジ内かどうかの値を定義。初期化し、全ての値を0（範囲外）に設定
df['is_in_cashier_zone'] = 0

# すべての行についてレジ内かどうか判定する
for i in range(len(df) - 1):
    r_toe_point_1 = [df['3D KeyPoints'].iloc[i][19][0], df['3D KeyPoints'].iloc[i][19][1]] # 右つま先xy座標（フレーム最初）
    l_toe_point_1 = [df['3D KeyPoints'].iloc[i][20][0], df['3D KeyPoints'].iloc[i][20][1]] # 左つま先xy座標
    r_toe_point_2 = [df['3D KeyPoints'].iloc[i+1][19][0], df['3D KeyPoints'].iloc[i+1][19][1]] # 右つま先xy座標（フレーム最後）
    l_toe_point_2 = [df['3D KeyPoints'].iloc[i+1][20][0], df['3D KeyPoints'].iloc[i+1][20][1]] # 左つま先xy座標

    if is_point_in_square(cashier_zone, r_toe_point_1) and is_point_in_square(cashier_zone, l_toe_point_1) and is_point_in_square(cashier_zone, r_toe_point_2) and is_point_in_square(cashier_zone, l_toe_point_2):
        df['is_in_cashier_zone'].iloc[i] = 1

# フレーム開始前と後のユークリッド距離を算出
# 'dis_kp'という新しい列をデータフレームに追加し、全ての値を空のリストとして初期化
df['dis_kp'] = [ [ ] for _ in range(len(df))]
# 最初の行から最後から2番目の行まででループ
for i in range(len(df) - 1):
    # 各キーポイントについてループ
    for j in range(len(df['3D KeyPoints'].iloc[i])):
        point1 = np.array(df['3D KeyPoints'].iloc[i][j][:2])
        point2 = np.array(df['3D KeyPoints'].iloc[i+1][j][:2])
        # ユークリッド距離を計算
        distance = np.linalg.norm(point1 - point2)
        # 計算した距離を'dis_kp'のi行j列に追加
        df['dis_kp'].iloc[i].append(distance)
# 最後の行は-1とする
df['dis_kp'].iloc[-1] = [-1] * 21

# 動いたか否かの閾値
# FIXME: チューニングの必要あり
D = 100

# 下記の3つを全て満たす行のみ、「会計」と判定
# 1. 「カートに追加」ではない  df['state'].iloc[i] != state_dict['カートに追加']
# 2. レジ前範囲内  df['is_in_cashier_zone'].iloc[i] == 1
# 3. 左つま先・右つま先がどちらもD以上動いていない  df['dis_kp'].iloc[i][19] <= D and df['dis_kp'].iloc[i][20] <= D 
for i in range(len(df)):
    if df['state'].iloc[i] != state_dict['カートに追加'] and df['is_in_cashier_zone'].iloc[i] == 1 and df['dis_kp'].iloc[i][19] <= D and df['dis_kp'].iloc[i][20] <= D:
        df['state'].iloc[i] = state_dict['会計']


# iii. 「移動」「ウェイト」の判定
for i in range(len(df)):
    if df['state'].iloc[i] == state_dict['未設定']:
        if df['dis_kp'].iloc[i][19] > D or df['dis_kp'].iloc[i][20] > D:
            df['state'].iloc[i] = state_dict['移動']
        else:
            df['state'].iloc[i] = state_dict['ウェイト']

# stateのデータを含んだCSVを出力
df.to_csv('data/tmp/state/' + input_tsv_name, index=False)


# stateを格納するためのリスト
statelist = df['state'].tolist()

print(statelist)

[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 4]


/var/folders/86/gydl1dsx32j_7v35v72dzvz40000gn/T/ipykernel_926/3834779669.py:49: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['dis_kp'].iloc[-1] = [-1] * 21
/var/folders/86/gydl1dsx32j_7v35v72dzvz40000gn/T/ipykernel_926/3834779669.py:49:

In [5]:
# 状態リスト→自然言語への変換
import csv
# 「移動」→ 1,「カートに追加」→ 2,「会計」→ 3,「ウェイト」→ 4
print(statelist)

# 結果保存用
result = []
result.append('初期化')
# 初期状態設定
prev_state = statelist[0]
count = 1

# 状態とカウント数から文字列を出力する関数
def print_state(state, count):
    str = ''
    if state == state_dict['移動']:
        str = f'移動,{count*0.5}秒'
    elif state == state_dict['カートに追加']:
        str = f'カートに追加,{count*0.5}秒'
    elif state == state_dict['会計']:
        str = f'会計,{count*0.5}秒'
    elif state == state_dict['ウェイト']:
        str = f'ウェイト,{count*0.5}秒'
    return str

# 同じ状態が連続している場合、秒数を加算して出力する
for state in statelist[1:]:
    if state == prev_state:
        count += 1
    else:
        result.append(print_state(prev_state, count))
        prev_state = state
        count = 1

result.append(print_state(prev_state, count))

print(result)

# ファイルに出力
# 出力ファイル名
output_file = './data/output/行動系列_' + input_tsv_name

with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    for row in result:
        # writerow関数でリストの内容をCSVに書き込む
        writer.writerow([row])

[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 4]
['初期化', '移動,3.5秒', 'カートに追加,4.0秒', '移動,1.0秒', 'ウェイト,0.5秒']
